# Let's Get Started with Spanner Graph for Identity Management

## Install all of the necessary Python Libraries

In [ ]:
!pip install -r requirements.txt

## Login with gcloud

In [ ]:
!gcloud auth application-default login

project_id = !(gcloud config get-value project)
if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

## Create and Load the Database

In [ ]:
!gcloud spanner instances create useridentity --description="UserIdentity Database" --config=regional-us-west1 --edition=ENTERPRISE --processing-units=100 --default-backup-schedule-type=NONE
!gcloud spanner databases create useridentitydb --instance  useridentity --ddl-file=UserIdentityDB.sql

## Load the data

In [ ]:
from load_data import LoadData
LoadData()

## Load the Spanner Graph Library

%load_ext spanner_graphs

## Run Some Queries

In [ ]:
### Find distinct emails per month

%%spanner_graph --project {project_id} --instance useridentity --database useridentitydb

GRAPH UserIdentity
MATCH (o:SalesOrder)-[h:HAS_EMAIL]->(e:Email)
RETURN  EXTRACT(YEAR FROM h.ts) AS Year, EXTRACT(MONTH FROM h.ts) AS Month,
COUNT(DISTINCT e.email) as Email ORDER BY Month

In [ ]:
### Draw all of the relationship of a specific email address in a graphical format

%%spanner_graph --project {project_id} --instance useridentity --database useridentitydb

GRAPH UserIdentity
MATCH p=(e:Email{email: "jamesparks@example.com" })-[h]->{1,3}(j)
WHERE h[0].ts > "2025-04-01"
RETURN SAFE_TO_JSON(p) AS JSON

In [ ]:
### Find Innocent looking emails that are linked through devices to suspicious emails

%%spanner_graph --project {project_id} --instance useridentity --database useridentitydb

GRAPH UserIdentity
MATCH (e:Email{sus: 0})-[h:HAS_DEVICE]->(d:Device)<-[h2:HAS_DEVICE]-(e2:Email{sus:1})
WHERE e.id != e2.id
RETURN e.email AS PossiblyBad, e2.email AS KnownBad

In [ ]:
### Which CCs are linked to the most suspicious transactions

%%spanner_graph --project {project_id} --instance useridentity --database useridentitydb

GRAPH UserIdentity
MATCH (e:Email{sus: 0})-[h:HAS_DEVICE]->(d:Device)<-[h2:HAS_DEVICE]-(e2:Email{sus:1})
WHERE e.id != e2.id
RETURN e.email AS PossiblyBad, e2.email AS KnownBad

In [ ]:
### When was the last time that we saw a transaction of this shape

%%spanner_graph --project {project_id} --instance useridentity --database useridentitydb

GRAPH UserIdentity
 MATCH (
  cc:CC{last4: "8963", zip: "36206"})<-[EMAIL_HAS_CC]-(
    e:Email{email:"kevin04@example.com"})<-[HAS_EMAIL]-(
      o:SalesOrder)-[hs:HAS_ADDRESS]->(sa:ShippingAddress)
 WHERE sa.id IN (
    SELECT id FROM ShippingAddress WHERE
    SEARCH_NGRAMS(address_Tokens, 'Hill AND Lodge AND 3608'))
 RETURN hs.ts AS TS, o.sus AS IS_SUSPECT

In [ ]:
### Find all Suspect Orders with and email and last4

%%spanner_graph --project {project_id} --instance useridentity --database useridentitydb

GRAPH UserIdentity
 MATCH (
  cc:CC{last4: "2218"})<-[EMAIL_HAS_CC]-(
    e:Email{email: "jeremy76@example.com"})<-[h:HAS_EMAIL]-(o:SalesOrder{sus: 1})
    WHERE h.ts > "2025-02-01"
 RETURN e.email AS EMAIL, cc.last4 AS LAST4, h.ts AS TS, o.id AS TRANSACT, o.sus AS IS_SUSPECT